<a href="https://colab.research.google.com/github/sdossou/LoRA_QLoRA/blob/main/%22Instruction_Tuning%22_Mistral_7B_Instruct_with_4_bit_QLoRA_vwikimovies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruct-tuning Mistral-7B-Instruct-v0.2 using `peft`, `transformers` and `bitsandbytes`

This notebook  fine-tunes Mistral-7B-Instruct-v0.2 a large language model.

This will retrain the weights of the model using QLoRA or Quantization Low-Rank Adaptation.

It uses the huggingface wiki_movies dataset to fine-tune Mistral-7B-Instruct-v0.2 to be able to generate instructions.





This  notebook focuses on the `bitsandbytes` quantization strategy which includes the idea of `k-bit` training.


### Dependencies

Installing relevant dependencies.

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━


### Model loading
Loading the `mistralai/Mistral-7B-Instruct-v0.2` model and tokenizer.

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
base_model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

In [ ]:
prompt = "The movie is"

In [ ]:
%time
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = base_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.95, temperature=0.4)
tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.34 µs


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'The movie is a 2005 American drama film directed by George Clooney, who also starred in it along with Brad Pitt, Sharon Stone, Sandra Bullock, and Julia Roberts. The film is about a group of Hollywood actors who travel to the Darfur region of Sudan to raise awareness about the ongoing humanitarian crisis there. The film received mixed reviews from critics, but it was a box office success.\n\nOne of the most memorable moments in the film comes when the'

Let's also load the tokenizer, along with the padding token.

In [ ]:

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#### Model Architecture

Applying LoRA to the modules related to the attention weights: `q_proj`, `v_proj`, `query_key_value`. This is model dependent

In [ ]:
print(base_model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

### Apply LoRA

Loading the `PeftModel` and specifying using low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

#### Helper Function to Print Parameter Percentage

This is just a helper function to print out just how much LoRA reduces the number of trainable parameters.

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Initialising LoRA Config
Main parameters:

- `r`: is the "rank" of the two decomposed matrices used to represent the weight matrix. This is the dimension of the decomposed matrices.

The following is an exerpt from the paper to help provide context for the selected `r`

- `target_modules`: As LoRA can be applied to *any* weight matrix, *which* module (weight matrix) it is being applied to needs to be configured. While the LoRA paper suggests applying it only to the Attention weights, this notebook follows the guidance of the QLoRA paper by applying LoRA to all Linear layers.


- `task_type`: This is a derived property. If you're using a causal model, this should be set to `CAUSAL_LM`. Ensure this property is set based on the selected model.


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    #target_modules=["q_proj", "v_proj", "k_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

base_model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, lora_config)
print_trainable_parameters(model)

trainable params: 27262976 || all params: 3779334144 || trainable%: 0.7213698223345028


In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024

### Preprocessing

Loading the wiki_movie dataset from Hugging Face.

In [ ]:
import transformers
from datasets import load_dataset

dataset_name = "wiki_movies"

In [ ]:
dataset = load_dataset(dataset_name)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 96185
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 9952
    })
    validation: Dataset({
        features: ['question', 'answer'],
        num_rows: 10000
    })
})


In [ ]:
print(dataset['train'][0])

{'question': ' what movies are about ginger rogers?', 'answer': 'Top Hat, Kitty Foyle, The Barkleys of Broadway\n'}


In [ ]:
print(dataset['train'][5])

{'question': ' what movies can be described with ben stiller?', 'answer': "Tropic Thunder, Meet the Parents, There's Something About Mary, Madagascar, The Secret Life of Walter Mitty, Night at the Museum, Meet the Fockers, The Royal Tenenbaums, Zoolander, The Cable Guy, Tower Heist, Along Came Polly, Little Fockers, The Heartbreak Kid, Keeping the Faith, Duplex, Reality Bites, Greenberg, Envy, Flirting with Disaster, Zero Effect\n"}


Limiting the number of samples to 5K.

In [ ]:
dataset_subset = dataset["train"].select(range(5_000))

Putting the data in the following form:

```
Generate a simple instruction an LLM could use to generate the provided context.
[INST]CONTEXT: 1.Top Hat, Kitty Foyle, The Barkleys of Broadway.
[/INST]INSTRUCTION: what movies are about ginger rogers?
```

 Showing the model examples of a prompt and its completion.

In [ ]:
def generate_prompt(example, return_response=True) -> str:
  full_prompt = f"Generate a simple question that could result in the provided context."
  full_prompt += f"[INST]CONTEXT: {example['answer']}[/INST]"

  if return_response:
    full_prompt += f"INSTRUCTION: "
    full_prompt += f"{example['question']}"
  return [full_prompt]

In [ ]:
generate_prompt(dataset_subset[0])[0]

'Generate a simple question that could result in the provided context.[INST]CONTEXT: Top Hat, Kitty Foyle, The Barkleys of Broadway\n[/INST]INSTRUCTION:  what movies are about ginger rogers?'

The `Trainer` class contains the same hyper-parameters yas traditional ML applications.

If you're running into CUDA memory issues - please modify both the `per_device_train_batch_size` to be lower, and also reduce `r` in the LoRAConfig. You will need to restart and re-run your notebook after doing so.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="mistral-7b-instruct",
    num_train_epochs=100,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit", # from the QLoRA paper
    logging_steps=1,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True, # ensure proper upcasting for compute dtypes
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True
)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install trl -U -q

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_subset,
    peft_config=lora_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=generate_prompt,
    args=training_args,
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

{'loss': 1.8399, 'grad_norm': 5.5662970542907715, 'learning_rate': 0.0002, 'epoch': 1.0}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.7438, 'grad_norm': 5.399608135223389, 'learning_rate': 0.0002, 'epoch': 2.0}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.5656, 'grad_norm': 1.5885920524597168, 'learning_rate': 0.0002, 'epoch': 3.0}


...

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.007, 'grad_norm': 0.1176724061369896, 'learning_rate': 0.0002, 'epoch': 99.0}


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.0071, 'grad_norm': 0.7315018773078918, 'learning_rate': 0.0002, 'epoch': 100.0}
{'train_runtime': 531.2239, 'train_samples_per_second': 0.941, 'train_steps_per_second': 0.188, 'train_loss': 0.27044895668514074, 'epoch': 100.0}


TrainOutput(global_step=100, training_loss=0.27044895668514074, metrics={'train_runtime': 531.2239, 'train_samples_per_second': 0.941, 'train_steps_per_second': 0.188, 'train_loss': 0.27044895668514074, 'epoch': 100.0})

In [ ]:
trainer.save_model()

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    training_args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(training_args.output_dir)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
sample = dataset_subset[5]

prompt = generate_prompt(sample, return_response=False)

In [ ]:
input_ids = tokenizer(prompt[0], return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(f"Prompt:\n{prompt[0]}\n")
print(f"-------------")
print(f"Generated question:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt[0]):]}")
print(f"-------------")
print(f"Ground truth:\n{sample['question']}")

Prompt:
Generate a simple question that could result in the provided context.[INST]CONTEXT: Tropic Thunder, Meet the Parents, There's Something About Mary, Madagascar, The Secret Life of Walter Mitty, Night at the Museum, Meet the Fockers, The Royal Tenenbaums, Zoolander, The Cable Guy, Tower Heist, Along Came Polly, Little Fockers, The Heartbreak Kid, Keeping the Faith, Duplex, Reality Bites, Greenberg, Envy, Flirting with Disaster, Zero Effect
[/INST]

-------------
Generated question:
 What are the names of the films that have been a part of Ben Stiller's acting and directing career, including his comedies like Tropic Thunder, Meet the Parents, and Zoolander?
-------------
Ground truth:
 what movies can be described with ben stiller?


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

untuned_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
input_ids = tokenizer(prompt[0], return_tensors="pt", truncation=True).input_ids.cuda()

outputs = untuned_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.5)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(f"Prompt:\n{prompt}\n")
print(f"-------------")
print(f"Generated question:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt[0]):]}")
print(f"-------------")
print(f"Ground truth:\n{sample['question']}")

Prompt:
["Generate a simple question that could result in the provided context.[INST]CONTEXT: Tropic Thunder, Meet the Parents, There's Something About Mary, Madagascar, The Secret Life of Walter Mitty, Night at the Museum, Meet the Fockers, The Royal Tenenbaums, Zoolander, The Cable Guy, Tower Heist, Along Came Polly, Little Fockers, The Heartbreak Kid, Keeping the Faith, Duplex, Reality Bites, Greenberg, Envy, Flirting with Disaster, Zero Effect\n[/INST]"]

-------------
Generated question:
 Which movies were released between the years 1998 and 2010 that feature in the given list?
-------------
Ground truth:
 what movies can be described with ben stiller?


This notebook is adapted from the notebook developed by AI Makerspace